### Data preprocessing

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest

##### Training data preprocessing

In [2]:
trainSet = pd.read_csv('Trainset.csv')
m = trainSet.shape[0]
features = list(trainSet.columns)[:]
trainSet.shape

(10480, 18)

In [3]:
Y = trainSet.iloc[:, 17:18].values 
X0 = trainSet.iloc[:, 10:17].values #month....weekend
X1 = trainSet.iloc[:, 0:1].values   #homepage
X2 = trainSet.iloc[:, 2:3].values   #aboutus
X3 = trainSet.iloc[:, 4:5].values   #contactus
X4 = trainSet.iloc[:, 1:2].values   #homepage_time
X5 = trainSet.iloc[:, 3:4].values   #aboutus_time
X6 = trainSet.iloc[:, 5:10].values   #contactus_time....special_day

In [4]:
imp_mf = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

imp_mf.fit(X0)
X0 = imp_mf.transform(X0)
imp_mf.fit(X1)
X1 = imp_mf.transform(X1)
imp_mf.fit(X2)
X2 = imp_mf.transform(X2)
imp_mf.fit(X3)
X3 = imp_mf.transform(X3)

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X4[X4 < 0] = np.nan
imp_mean.fit(X4)
X4 = imp_mean.transform(X4)
X5[X5 < 0] = np.nan
imp_mean.fit(X5)
X5 = imp_mean.transform(X5)
X6[X6 < 0] = np.nan
imp_mean.fit(X6)
X6 = imp_mean.transform(X6)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in less
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in less


In [5]:
X= np.append(X1,X4,axis=1)
X= np.append(X,X2,axis=1)
X= np.append(X,X5,axis=1)
X= np.append(X,X3,axis=1)
X= np.append(X,X6,axis=1)
X= np.append(X,X0,axis=1)
df= np.append(X,Y,axis=1)
df.shape

(10480, 18)

In [6]:
dfObj = pd.DataFrame(df, columns = ['Homepage', 'Homepage _Duration', 'Aboutus', 'Aboutus_Duration',
       'Contactus', 'Contactus_Duration', 'BounceRates', 'ExitRates',
       'PageValues', 'SpecialDay', 'Month', 'OperatingSystems', 'Browser',
       'Province','TrafficType', 'VisitorType', 'Weekend','Revenue']) 
dfObj.to_csv('imputedData.csv', index=False)

In [7]:
trainSet = pd.read_csv('imputedData.csv')

In [8]:
month = []
visitorType = []
weekend = []
province = []
browser = []
operatingSystems = []
for i in features:
    if i == 'Month':
        columData = list(trainSet[i])
        #print (columData)
        for j in range(m):
            if columData[j] == 'Jan':
                month.append([1,0,0,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 'Feb':
                month.append([0,1,0,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 'Mar':
                month.append([0,0,1,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 'Apr':
                month.append([0,0,0,1,0,0,0,0,0,0,0,0])
            elif columData[j] == 'May':
                month.append([0,0,0,0,1,0,0,0,0,0,0,0])
            elif columData[j] == 'June':
                month.append([0,0,0,0,0,1,0,0,0,0,0,0])
            elif columData[j] == 'Jul':
                month.append([0,0,0,0,0,0,1,0,0,0,0,0])
            elif columData[j] == 'Aug':
                month.append([0,0,0,0,0,0,0,1,0,0,0,0])
            elif columData[j] == 'Sep':
                month.append([0,0,0,0,0,0,0,0,1,0,0,0])
            elif columData[j] == 'Oct':
                month.append([0,0,0,0,0,0,0,0,0,1,0,0])
            elif columData[j] == 'Nov':
                month.append([0,0,0,0,0,0,0,0,0,0,1,0])
            elif columData[j] == 'Dec':
                month.append([0,0,0,0,0,0,0,0,0,0,0,1])
            else:
                print (1)
    elif i == 'Weekend':
        columData = list(trainSet[i])
        for j in range(m):
            if columData[j] == True:
                weekend.append([1,0])
            elif columData[j] == False:
                weekend.append([0,1])             
    elif i == 'VisitorType':
        columData = list(trainSet[i])
        for j in range(m):
            if columData[j] == 'New_Visitor':
                visitorType.append([1,0,0])
            elif columData[j] == 'Returning_Visitor':
                visitorType.append([0,1,0])
            elif columData[j] == 'Other':
                visitorType.append([0,0,1])
            else:
                print (columData[j])
    elif i == 'Province':
        columData = list(trainSet[i])
        for j in range(m):
            if columData[j] == 1:
                province.append([1,0,0,0,0,0,0,0,0])
            elif columData[j] == 2:
                province.append([0,1,0,0,0,0,0,0,0])
            elif columData[j] == 3:
                province.append([0,0,1,0,0,0,0,0,0])
            elif columData[j] == 4:
                province.append([0,0,0,1,0,0,0,0,0])
            elif columData[j] == 5:
                province.append([0,0,0,0,1,0,0,0,0])
            elif columData[j] == 6:
                province.append([0,0,0,0,0,1,0,0,0])
            elif columData[j] == 7:
                province.append([0,0,0,0,0,0,1,0,0])
            elif columData[j] == 8:
                province.append([0,0,0,0,0,0,0,1,0])
            elif columData[j] == 9:
                province.append([0,0,0,0,0,0,0,0,1])
            else:
                print (columData[j])
    elif i == 'Browser':
        columData = list(trainSet[i])
        for j in range(m):
            if columData[j] == 1:
                browser.append([1,0,0,0,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 2:
                browser.append([0,1,0,0,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 3:
                browser.append([0,0,1,0,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 4:
                browser.append([0,0,0,1,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 5:
                browser.append([0,0,0,0,1,0,0,0,0,0,0,0,0])
            elif columData[j] == 6:
                browser.append([0,0,0,0,0,1,0,0,0,0,0,0,0])
            elif columData[j] == 7:
                browser.append([0,0,0,0,0,0,1,0,0,0,0,0,0])
            elif columData[j] == 8:
                browser.append([0,0,0,0,0,0,0,1,0,0,0,0,0])
            elif columData[j] == 9:
                browser.append([0,0,0,0,0,0,0,0,1,0,0,0,0])
            elif columData[j] == 10:
                browser.append([0,0,0,0,0,0,0,0,0,1,0,0,0])
            elif columData[j] == 11:
                browser.append([0,0,0,0,0,0,0,0,0,0,1,0,0])
            elif columData[j] == 12:
                browser.append([0,0,0,0,0,0,0,0,0,0,0,1,0])
            elif columData[j] == 13:
                browser.append([0,0,0,0,0,0,0,0,0,0,0,0,1])
            else:
                print (columData[j])
    elif i == 'OperatingSystems':
        columData = list(trainSet[i])
        for j in range(m):
            if columData[j] == 1:
                operatingSystems.append([1,0,0,0,0,0,0,0])
            elif columData[j] == 2:
                operatingSystems.append([0,1,0,0,0,0,0,0])
            elif columData[j] == 3:
                operatingSystems.append([0,0,1,0,0,0,0,0])
            elif columData[j] == 4:
                operatingSystems.append([0,0,0,1,0,0,0,0])
            elif columData[j] == 5:
                operatingSystems.append([0,0,0,0,1,0,0,0])
            elif columData[j] == 6:
                operatingSystems.append([0,0,0,0,0,1,0,0])
            elif columData[j] == 7:
                operatingSystems.append([0,0,0,0,0,0,1,0])
            elif columData[j] == 8:
                operatingSystems.append([0,0,0,0,0,0,0,1])
            else:
                print (columData[j])


In [9]:
X_0 = trainSet.iloc[:, 0:10].values
X_1 = trainSet.iloc[:, 14:15].values    #traffic type
Y_ = trainSet.iloc[:, 17:18].values
X_ = np.append(X_0,month,axis=1)
X_ = np.append(X_,operatingSystems,axis=1)
X_ = np.append(X_,browser,axis=1)
X_ = np.append(X_,province,axis=1)
X_ = np.append(X_,X_1,axis=1)
X_ = np.append(X_,visitorType,axis=1)
X_ = np.append(X_,weekend,axis=1)
df_ = np.append(X_,Y_,axis=1)
df_.shape

(10480, 59)

In [10]:
featureNames = ['homepage', 'homepage _duration', 'aboutus', 'aboutus_duration',
       'contactus', 'contactus_duration', 'bounceRates', 'exitRates',
       'PageValues', 'SpecialDay','Month1','Month2','Month3','Month4','Month5','Month6','Month7','Month8',
        'Month9','Month10','Month11','Month12','os1','os2','os3','os4','os5','os6','os7','os8',
        'browser1','browser2','browser3','browser4','browser5','browser6','browser7','browser8','browser9',
        'browser10','browser11','browser12','browser13','province1','province2','province3','province4','province5',
        'province6','province7','province8','province9','Traffic_Type','visitor1','visitor2','visitor3',
        'weekend1','weekwnd2','Revenue']
print (len(featureNames))

59


In [11]:
dfObj = pd.DataFrame(df_, columns = featureNames) 
dfObj.to_csv('oneHotData.csv', index=False)

##### Test data preprocessing

In [12]:
testSet = pd.read_csv('xtest.csv')
m = testSet.shape[0]
features = list(testSet.columns)[:]
testSet = testSet.iloc[:, 1:18].values
testSet.shape
testSet= pd.DataFrame(testSet, columns = ['Homepage', 'Homepage _Duration', 'Aboutus', 'Aboutus_Duration',
      'Contactus', 'Contactus_Duration', 'BounceRates', 'ExitRates',
      'PageValues', 'SpecialDay','TrafficType', 'Month', 'OperatingSystems', 'Browser',
      'Province', 'VisitorType', 'Weekend']) 
testSet.shape

(1850, 17)

In [13]:
X0 = testSet.iloc[:, 10:17].values #month....weekend
X1 = testSet.iloc[:, 0:1].values   #homepage
X2 = testSet.iloc[:, 2:3].values   #aboutus
X3 = testSet.iloc[:, 4:5].values   #contactus
X4 = testSet.iloc[:, 1:2].values   #homepage_time
X5 = testSet.iloc[:, 3:4].values   #aboutus_time
X6 = testSet.iloc[:, 5:10].values   #contactus_time....special_day

In [14]:
imp_mf = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

imp_mf.fit(X0)
X0 = imp_mf.transform(X0)
imp_mf.fit(X1)
X1 = imp_mf.transform(X1)
imp_mf.fit(X2)
X2 = imp_mf.transform(X2)
imp_mf.fit(X3)
X3 = imp_mf.transform(X3)

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X4[X4 < 0] = np.nan
imp_mean.fit(X4)
X4 = imp_mean.transform(X4)
X5[X5 < 0] = np.nan
imp_mean.fit(X5)
X5 = imp_mean.transform(X5)
X6[X6 < 0] = np.nan
imp_mean.fit(X6)
X6 = imp_mean.transform(X6)

In [15]:
X= np.append(X1,X4,axis=1)
X= np.append(X,X2,axis=1)
X= np.append(X,X5,axis=1)
X= np.append(X,X3,axis=1)
X= np.append(X,X6,axis=1)
X= np.append(X,X0,axis=1)
df= X
df.shape

(1850, 17)

In [16]:
dfObj = pd.DataFrame(df, columns = ['Homepage', 'Homepage _Duration', 'Aboutus', 'Aboutus_Duration',
       'Contactus', 'Contactus_Duration', 'BounceRates', 'ExitRates',
       'PageValues', 'SpecialDay', 'Month', 'OperatingSystems', 'Browser',
       'Province','TrafficType', 'VisitorType', 'Weekend']) 
dfObj.to_csv('imputedTestData.csv', index=False)

In [17]:
trainSet = pd.read_csv('imputedTestData.csv')

In [18]:
month = []
visitorType = []
weekend = []
province = []
browser = []
operatingSystems = []
for i in features:
    if i == 'Month':
        columData = list(trainSet[i])
        #print (columData)
        for j in range(m):
            if columData[j] == 'Jan':
                month.append([1,0,0,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 'Feb':
                month.append([0,1,0,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 'Mar':
                month.append([0,0,1,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 'Apr':
                month.append([0,0,0,1,0,0,0,0,0,0,0,0])
            elif columData[j] == 'May':
                month.append([0,0,0,0,1,0,0,0,0,0,0,0])
            elif columData[j] == 'June':
                month.append([0,0,0,0,0,1,0,0,0,0,0,0])
            elif columData[j] == 'Jul':
                month.append([0,0,0,0,0,0,1,0,0,0,0,0])
            elif columData[j] == 'Aug':
                month.append([0,0,0,0,0,0,0,1,0,0,0,0])
            elif columData[j] == 'Sep':
                month.append([0,0,0,0,0,0,0,0,1,0,0,0])
            elif columData[j] == 'Oct':
                month.append([0,0,0,0,0,0,0,0,0,1,0,0])
            elif columData[j] == 'Nov':
                month.append([0,0,0,0,0,0,0,0,0,0,1,0])
            elif columData[j] == 'Dec':
                month.append([0,0,0,0,0,0,0,0,0,0,0,1])
            else:
                print (1)
    elif i == 'Weekend':
        columData = list(trainSet[i])
        for j in range(m):
            if columData[j] == True:
                weekend.append([1,0])
            elif columData[j] == False:
                weekend.append([0,1])             
    elif i == 'VisitorType':
        columData = list(trainSet[i])
        for j in range(m):
            if columData[j] == 'New_Visitor':
                visitorType.append([1,0,0])
            elif columData[j] == 'Returning_Visitor':
                visitorType.append([0,1,0])
            elif columData[j] == 'Other':
                visitorType.append([0,0,1])
            else:
                print (columData[j])
    elif i == 'Province':
        columData = list(trainSet[i])
        for j in range(m):
            if columData[j] == 1:
                province.append([1,0,0,0,0,0,0,0,0])
            elif columData[j] == 2:
                province.append([0,1,0,0,0,0,0,0,0])
            elif columData[j] == 3:
                province.append([0,0,1,0,0,0,0,0,0])
            elif columData[j] == 4:
                province.append([0,0,0,1,0,0,0,0,0])
            elif columData[j] == 5:
                province.append([0,0,0,0,1,0,0,0,0])
            elif columData[j] == 6:
                province.append([0,0,0,0,0,1,0,0,0])
            elif columData[j] == 7:
                province.append([0,0,0,0,0,0,1,0,0])
            elif columData[j] == 8:
                province.append([0,0,0,0,0,0,0,1,0])
            elif columData[j] == 9:
                province.append([0,0,0,0,0,0,0,0,1])
            else:
                print (columData[j])
    elif i == 'Browser':
        columData = list(trainSet[i])
        for j in range(m):
            if columData[j] == 1:
                browser.append([1,0,0,0,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 2:
                browser.append([0,1,0,0,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 3:
                browser.append([0,0,1,0,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 4:
                browser.append([0,0,0,1,0,0,0,0,0,0,0,0,0])
            elif columData[j] == 5:
                browser.append([0,0,0,0,1,0,0,0,0,0,0,0,0])
            elif columData[j] == 6:
                browser.append([0,0,0,0,0,1,0,0,0,0,0,0,0])
            elif columData[j] == 7:
                browser.append([0,0,0,0,0,0,1,0,0,0,0,0,0])
            elif columData[j] == 8:
                browser.append([0,0,0,0,0,0,0,1,0,0,0,0,0])
            elif columData[j] == 9:
                browser.append([0,0,0,0,0,0,0,0,1,0,0,0,0])
            elif columData[j] == 10:
                browser.append([0,0,0,0,0,0,0,0,0,1,0,0,0])
            elif columData[j] == 11:
                browser.append([0,0,0,0,0,0,0,0,0,0,1,0,0])
            elif columData[j] == 12:
                browser.append([0,0,0,0,0,0,0,0,0,0,0,1,0])
            elif columData[j] == 13:
                browser.append([0,0,0,0,0,0,0,0,0,0,0,0,1])
            else:
                print (columData[j])
    elif i == 'OperatingSystems':
        columData = list(trainSet[i])
        for j in range(m):
            if columData[j] == 1:
                operatingSystems.append([1,0,0,0,0,0,0,0])
            elif columData[j] == 2:
                operatingSystems.append([0,1,0,0,0,0,0,0])
            elif columData[j] == 3:
                operatingSystems.append([0,0,1,0,0,0,0,0])
            elif columData[j] == 4:
                operatingSystems.append([0,0,0,1,0,0,0,0])
            elif columData[j] == 5:
                operatingSystems.append([0,0,0,0,1,0,0,0])
            elif columData[j] == 6:
                operatingSystems.append([0,0,0,0,0,1,0,0])
            elif columData[j] == 7:
                operatingSystems.append([0,0,0,0,0,0,1,0])
            elif columData[j] == 8:
                operatingSystems.append([0,0,0,0,0,0,0,1])
            else:
                print (columData[j])

In [19]:
X_0 = trainSet.iloc[:, 0:10].values
X_1 = trainSet.iloc[:, 14:15].values    #traffic type
X_ = np.append(X_0,month,axis=1)
X_ = np.append(X_,operatingSystems,axis=1)
X_ = np.append(X_,browser,axis=1)
X_ = np.append(X_,province,axis=1)
X_ = np.append(X_,X_1,axis=1)
X_ = np.append(X_,visitorType,axis=1)
X_ = np.append(X_,weekend,axis=1)
df_ = X_
df_.shape

(1850, 58)

In [20]:
featureNames = ['homepage', 'homepage _duration', 'aboutus', 'aboutus_duration',
       'contactus', 'contactus_duration', 'bounceRates', 'exitRates',
       'PageValues', 'SpecialDay','Month1','Month2','Month3','Month4','Month5','Month6','Month7','Month8',
        'Month9','Month10','Month11','Month12','os1','os2','os3','os4','os5','os6','os7','os8',
        'browser1','browser2','browser3','browser4','browser5','browser6','browser7','browser8','browser9',
        'browser10','browser11','browser12','browser13','province1','province2','province3','province4','province5',
        'province6','province7','province8','province9','Traffic_Type','visitor1','visitor2','visitor3',
        'weekend1','weekwnd2']
print (len(featureNames))

58


In [21]:
dfObj = pd.DataFrame(df_, columns = featureNames) 
dfObj.to_csv('oneHotTestData.csv', index=False)

### Training models

In [22]:
trainData = pd.read_csv('oneHotData.csv')
print (trainData.shape)
m = trainData.shape[0]
features = list(trainData.columns)[:]
Y = trainData[features[-1]]
X = trainData[features[:]]
#trainData[features[52:56]]

(10480, 59)


In [23]:
#select 12 features from dataset
X_0 = trainData.iloc[:, 0:9].values
X_1 = trainData.iloc[:, 20:21].values
X_2 = trainData.iloc[:, 52:54].values
X = np.append(X_0,X_1, axis = 1)
X = np.append(X,X_2, axis = 1)
X = pd.DataFrame(X)
X.shape

(10480, 12)

In [25]:
clf = IsolationForest(n_estimators=100)#warm_start=True)
Out= clf.fit_predict(X)  # fit 10 trees 
Out2= clf.score_samples(X)

Out2= Out2*(-1)
Out2= Out2.transpose()
Out2= np.reshape(Out2,(m,1))
X= np.append(X,Out2, axis=1)

Out= Out*(-1)
Out= Out.transpose()
Out= np.reshape(Out,(m,1))
X= np.append(X,Out, axis=1)
X.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


(10480, 14)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_train = X
y_train = Y
X1_train = (pd.DataFrame(X_train)).iloc[:, 0:10].values
X2_train = (pd.DataFrame(X_train)).iloc[:, 2:].values
X1_test = (pd.DataFrame(X_test)).iloc[:, 0:10].values
X2_test = (pd.DataFrame(X_test)).iloc[:, 2:].values
print (X1_train.shape)
print (X2_train.shape)

(10480, 10)
(10480, 12)


In [27]:
regressor1 = RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth= 9,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators=300,
                                                    n_jobs=None,)
regressor2 = RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth= 9,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators=300,
                                                    n_jobs=None,)
regressor1.fit(X1_train, y_train)
regressor2.fit(X2_train, y_train)
print ('Done')

Done


In [28]:
y_pred1 = regressor1.predict(X1_train)
y_predtest1 = regressor1.predict(X1_test)
y_pred2 = regressor2.predict(X2_train)
y_predtest2 = regressor2.predict(X2_test)

In [29]:
print(accuracy_score(y_train, y_pred1))
print(accuracy_score(y_test, y_predtest1))
print(accuracy_score(y_train, y_pred2))
print(accuracy_score(y_test, y_predtest2))

0.9412213740458015
0.94529262086514
0.9361641221374046
0.9392493638676844


In [30]:
XL = []
for i in range(y_pred1.shape[0]):
    row  = []
    row.append(y_pred1[i])
    row.append(y_pred2[i])
    XL.append(row)
XL = np.array(XL)
XL = np.append(X1_train, XL, axis = 1)
XL = np.append(X2_train, XL, axis = 1)
XL = pd.DataFrame(XL)
XL.shape

(10480, 24)

In [31]:
XLtest = []
for i in range(y_predtest1.shape[0]):
    row  = []
    row.append(y_predtest1[i])
    row.append(y_predtest2[i])
    XLtest.append(row)
XLtest = np.array(XLtest)
XLtest = np.append(X1_test, XLtest, axis = 1)
XLtest = np.append(X2_test, XLtest, axis = 1)
XLtest = pd.DataFrame(XLtest)
XLtest.shape

(3144, 24)

In [32]:
#Create a Gaussian Classifier
regressor3=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth= 9,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators=300,
                                                    n_jobs=None,)

regressor3.fit(XL, y_train)
print ('Done')

Done


In [33]:
y_pred = regressor3.predict(XLtest)
y_train_pred = regressor3.predict(XL)

In [45]:
print("train accuracy= ",accuracy_score(y_train, y_train_pred))
print("test accuracy= ", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

train accuracy=  0.9460877862595419
test accuracy=  0.9494274809160306
[[2608   33]
 [ 126  377]]
              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97      2641
         1.0       0.92      0.75      0.83       503

   micro avg       0.95      0.95      0.95      3144
   macro avg       0.94      0.87      0.90      3144
weighted avg       0.95      0.95      0.95      3144



In [35]:
print(accuracy_score(y_test, y_pred))
print(accuracy_score(y_train, y_train_pred))

0.9494274809160306
0.9460877862595419


### Predict on xtest.csv

In [36]:
testData = pd.read_csv('oneHotTestData.csv')
mTest = testData.shape[0]
featuresTest = list(testData.columns)[:]
XT = testData[featuresTest[:]]
#select 12 features from dataset
X_0 = testData.iloc[:, 0:9].values
X_1 = testData.iloc[:, 20:21].values
X_2 = testData.iloc[:, 52:54].values
XT = np.append(X_0,X_1, axis = 1)
XT = np.append(XT,X_2, axis = 1)
XT = pd.DataFrame(XT)
XT.shape

(1850, 12)

In [37]:
clf = IsolationForest(n_estimators=100)# warm_start=True)
Out= clf.fit_predict(XT)  # fit 10 trees 
Out2= clf.score_samples(XT)

Out2= Out2*(-1)
Out2= Out2.transpose()
Out2= np.reshape(Out2,(1850,1))
XT= np.append(XT,Out2, axis=1)

Out= Out*(-1)
Out= Out.transpose()
Out= np.reshape(Out,(1850,1))
XT= np.append(XT,Out, axis=1)
XT.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:213: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:223: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:417: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


(1850, 14)

In [38]:
XT1 = (pd.DataFrame(XT)).iloc[:, 0:10].values
XT2 = (pd.DataFrame(XT)).iloc[:, 2:].values
print (XT1.shape)
print (XT2.shape)

(1850, 10)
(1850, 12)


In [39]:
y_pred1 = regressor1.predict(XT1)
y_pred2 = regressor2.predict(XT2)

In [40]:
XTL = []
for i in range(y_pred1.shape[0]):
    row  = []
    row.append(y_pred1[i])
    row.append(y_pred2[i])
    XTL.append(row)
XTL = np.array(XTL)
XTL = np.append(XT1, XTL, axis = 1)
XTL = np.append(XT2, XTL, axis = 1)
XTL = pd.DataFrame(XTL)
XTL.shape

(1850, 24)

In [41]:
y_predtest = regressor3.predict(XTL)

In [42]:
ones = 0
zeros = 0
predictions = []
for i in range(y_predtest.shape[0]):
    if y_predtest[i] == 1.00:
        predictions.append(1)
        
    else:
        predictions.append(0)
for i in range(len(predictions)):
    if predictions[i] ==1:
        ones +=1
    else:
        zeros +=1
print (ones)
print (zeros)

222
1628


In [43]:
submissionObj = pd.DataFrame(predictions, columns = ['Revenue']) 
submissionObj.index.name='ID'
submissionObj.index+=1
submissionObj.to_csv('submission.csv')